In [1]:
class Security(object):
    def __init__(self, symbol, price):
        self.symbol = symbol
        self.price = price
    
    def __repr__(self):
        return '{}: {}'.format(self.symbol, self.price)
    
class Trade(object):
    def __init__(self, security, amount, strategy):
        self.security = security
        self.amount = amount
        self.strategy = strategy
    
    def __repr__(self):
        return '{}, {}, {}'.format(self.security.symbol, self.strategy, self.amount)
    
class Broker(object):
    def __init__(self, name):
        self.name = name
        self.cash = 0
        self.shares = {}
    
    def __repr__(self):
        return '{}, Cash: {}'.format(self.name, str(self.cash))
    
    def trade(self, trade):
        if not trade.strategy in ['BUY', 'SELL']:
            raise Exception('{}: strategy undefined'.format(trade.strategy))
        
        if trade.strategy == 'BUY':
            # If we are buying, create a record in the broker DB or add the amoun
            cost = trade.security.price * trade.amount
            self.cash -= cost
            if not trade.security in self.shares:
                self.shares[trade.security.symbol] = 0
            self.shares[trade.security.symbol] += trade.amount
        else:
            # If we are selling, subtract the amount from the record
            if not trade.security.symbol in self.shares:
                raise Exception('Not holding {}'.format(trade.security))
            
            if self.shares[trade.security.symbol] < trade.amount:
                raise Exception(
                    'Insufficient shares of {}. Currently holding {}'.format(
                        trade.security.symbol, self.shares[trade.security.symbol]
                ))
            
            cost = trade.security.price * trade.amount
            self.cash += cost
            self.shares[trade.security.symbol] -= trade.amount

class Position(object):
    def __init__(self, security, amount, broker):
        self.security = security
        self.amount = amount
        self.broker = broker
    
    def __repr__(self):
        return '{}, {}: {}'.format(self.broker, self.security, self.amount)

# Keed track of all the positions we are trading now
positions = {}

def transact(security, quantity, strategy, broker):
    trade = Trade(security, quantity, strategy)
    broker.trade(trade)
    
    if not security.symbol in positions:
        positions[security.symbol] = Position(security, quantity, broker)
    else:
        position = positions[security.symbol]
        if strategy == 'BUY':
            position.amount += quantity
        else:
            position.amount -= quantity

In [6]:
def pick_best_broker_buy(security, quantity, brokers):
    # Pick the broker with the highest amount of cash and work downwards
    # Sort the list of brokers by their cash
    # Never let the cash of a broker go down to 0
    total_cash = sum((broker.cash - security.price) for broker in brokers)
    cost = security.price * quantity
    if cost > total_cash:
        raise Exception('Not enough cash to transact')
    
    brokers.sort(key=lambda x: x.cash, reverse=True)
    left_quantity = quantity
    count = 0
    while left_quantity and count < len(brokers):
        # While we still have shares to buy and we haven't run out of brokers
        broker = brokers[count]
        left_cost = left_quantity * security.price
        broker_quantity = left_quantity
        if left_cost >= broker.cash:
            # To ensure cash does not go down to 1, always trade 1 quantity less
            broker_quantity = (broker.cash//security.price) - 1
        
        # Make the transaction
        transact(security, broker_quantity, 'BUY', broker)
        # Log
        print('Bought {} shares from {}'.format(str(broker_quantity), broker.name))
        # Update the counts
        left_quantity -= broker_quantity        
        count += 1
    
def test_pick_best_broker_buy():
    # Test Harness
    aapl = Security('AAPL', 100)
    a = Broker('A')
    b = Broker('B')
    c = Broker('C')
    
    brokers = [a, b, c]
    a.cash = 1000
    b.cash = 2000
    c.cash = 5000
    pick_best_broker_buy(aapl, 70, brokers)

def pick_best_broker_sell(security, quantity, brokers):
    # Pick the broker with the lowest number of shares and work upwards
    # Sort the list of brokers by their shares of the security
    # Work only with brokers that hold this tsecurity
    brokers_with_security = list(filter(lambda x: security.symbol in x.shares, brokers))
    total_shares_owned = sum(broker.shares[security.symbol] for broker in brokers_with_security)
    if quantity > total_shares_owned:
        raise Exception('Not enough shares to sell')
    left_quantity = quantity
    count = 0
    # While we still have shares to sell and we haven't run out of brokers
    while left_quantity and count < len(brokers):
        broker = brokers[count]
        # The quantity to sell is the minimum of shares available to sell for the broker
        # Or the total shares to sell
        broker_quantity = min(broker.shares[security.symbol], left_quantity)
        # Make the transaction
        transact(security, broker_quantity, 'SELL', broker)
        # Log
        print('Sold {} shares from {}'.format(str(broker_quantity), broker.name))
        # Update the counts
        left_quantity -= broker_quantity
        count += 1    

def test_pick_best_broker_sell():
    # Test Harness
    aapl = Security('AAPL', 100)
    a = Broker('A')
    b = Broker('B')
    c = Broker('C')
    d = Broker('D')

    brokers = [a, b, c, d]
    a.cash = 1000
    b.cash = 2000
    c.cash = 5000
    d.cash = 10000
    transact(aapl, 5, 'BUY', a)
    transact(aapl, 10, 'BUY', b)
    transact(aapl, 20, 'BUY', c)

    pick_best_broker_sell(aapl, 20, brokers)

test_pick_best_broker_buy()
print('-----------------------')
test_pick_best_broker_sell()

Bought 49 shares from C
Bought 19 shares from B
Bought 2 shares from A
-----------------------
Sold 5 shares from A
Sold 10 shares from B
Sold 5 shares from C


In [8]:
pwd

'/Users/duttk/algo-practice'